In [1]:
from text2vec import SentenceModel
from tqdm.autonotebook import tqdm
from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd
import numpy as np
import re

t2v_model = SentenceModel("shibing624/text2vec-base-chinese", device="cuda")


/home/zhouyunmeng/software/Anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/zhouyunmeng/software/Anaconda3/lib/python3.9/site-packages/text2vec/utils/get_file.py:16: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
2023-05-29 13:59:46.781 | DEBUG    | text2vec.sentence_model:__init__:74 - Use device: cuda


In [2]:
quality_qa_list = []
quality_qa_list.append({'Q': '你这个比电话卡贵很多/太贵了/价格太高了',
                        'A': '我们产品不仅可以解决您封号的问题，还能极大提升销售和销售管理的效率；目前我们公司已合作了5万多家客户，还拿到了8千万的融资，产品的稳定性您可以放心。我们给您带来的不仅是稳定的外呼，拨打量翻倍一个人干两个人的活，还能严格的管理团队，让您团队业绩更好。并且在同等实力的公司里面，我们没有年费和开通门槛，1个坐席起卖；还有专门的销售专家团队，一起帮您提高销售效率；您可以免费试用一下我们的产品，看是否能帮到您；'})
quality_qa_list.append({'Q': '为什么要收资料？怎么要这么多资料？别人家不需要收这么多资料？这么多资料太麻烦了',
                        'A': '当前封号严重，就是因为国家在对外呼行业进行严格管控。要想不封号只有一条路径就是开通外呼专线，国家和运营商对开通外呼专线的基本要求是要提供企业资质，也就是企业要证明自己是合规合法的，例如营业执照、法人信息等企业资质。正规外呼专线都要求提供公司完整的资料，但凡不要求你提供的那都是假的，国家不可能允许，在合规的前提下我们公司所需要配合提供的资质是最少的；'})
quality_qa_list.append({'Q': '你们是怎么做封号的? 防封的原理是什么？封号的机制是什么？',
                        'A': '首先是高频拨打封号，其次是投诉举报，比如辱骂客户，长期拨打同号码，产生骚扰导致投诉。这两种情况都会封号。我们怎么解决的跟您说一下：首先用企业专用营销线路可以解决高频外呼封号，其次，投诉举报封号是工信部进行封号的，谁也解决不了。我们产品有电销卫士这个功能，可以帮您提前筛除各类风险号码，跳过该类号码拨打，从而降低投诉举报风险。'})

quality_qa_list


[{'Q': '你这个比电话卡贵很多/太贵了/价格太高了',
  'A': '我们产品不仅可以解决您封号的问题，还能极大提升销售和销售管理的效率；目前我们公司已合作了5万多家客户，还拿到了8千万的融资，产品的稳定性您可以放心。我们给您带来的不仅是稳定的外呼，拨打量翻倍一个人干两个人的活，还能严格的管理团队，让您团队业绩更好。并且在同等实力的公司里面，我们没有年费和开通门槛，1个坐席起卖；还有专门的销售专家团队，一起帮您提高销售效率；您可以免费试用一下我们的产品，看是否能帮到您；'},
 {'Q': '为什么要收资料？怎么要这么多资料？别人家不需要收这么多资料？这么多资料太麻烦了',
  'A': '当前封号严重，就是因为国家在对外呼行业进行严格管控。要想不封号只有一条路径就是开通外呼专线，国家和运营商对开通外呼专线的基本要求是要提供企业资质，也就是企业要证明自己是合规合法的，例如营业执照、法人信息等企业资质。正规外呼专线都要求提供公司完整的资料，但凡不要求你提供的那都是假的，国家不可能允许，在合规的前提下我们公司所需要配合提供的资质是最少的；'},
 {'Q': '你们是怎么做封号的? 防封的原理是什么？封号的机制是什么？',
  'A': '首先是高频拨打封号，其次是投诉举报，比如辱骂客户，长期拨打同号码，产生骚扰导致投诉。这两种情况都会封号。我们怎么解决的跟您说一下：首先用企业专用营销线路可以解决高频外呼封号，其次，投诉举报封号是工信部进行封号的，谁也解决不了。我们产品有电销卫士这个功能，可以帮您提前筛除各类风险号码，跳过该类号码拨打，从而降低投诉举报风险。'}]

In [3]:
quality_qa_list[1]['A']

'当前封号严重，就是因为国家在对外呼行业进行严格管控。要想不封号只有一条路径就是开通外呼专线，国家和运营商对开通外呼专线的基本要求是要提供企业资质，也就是企业要证明自己是合规合法的，例如营业执照、法人信息等企业资质。正规外呼专线都要求提供公司完整的资料，但凡不要求你提供的那都是假的，国家不可能允许，在合规的前提下我们公司所需要配合提供的资质是最少的；'

In [4]:
# 将quality_qa_list转化为dataframe
quality_qa_df = pd.DataFrame(quality_qa_list)
Q_embedding = []

for i in range(quality_qa_df.shape[0]):
    temp = []
    # 将Q用'?'和'？'和'/'分割
    for y in re.split(r'[?？/]', quality_qa_df['Q'][i]):
        temp.append(t2v_model.encode(y))
    # 将temp作为一个元素添加到Q_embedding中
    Q_embedding.append(temp)

quality_qa_df['Q_embedding'] = np.array(Q_embedding, dtype=object)

quality_qa_df


,Q,A,Q_embedding
0,你这个比电话卡贵很多/太贵了/价格太高了,我们产品不仅可以解决您封号的问题，还能极大提升销售和销售管理的效率；目前我们公司已合作了5万...,"[[0.07692516, 0.22091311, 0.7793291, 1.1053766..."
1,为什么要收资料？怎么要这么多资料？别人家不需要收这么多资料？这么多资料太麻烦了,当前封号严重，就是因为国家在对外呼行业进行严格管控。要想不封号只有一条路径就是开通外呼专线，...,"[[-0.47177145, 1.3111272, 1.2139087, 0.1688726..."
2,你们是怎么做封号的? 防封的原理是什么？封号的机制是什么？,首先是高频拨打封号，其次是投诉举报，比如辱骂客户，长期拨打同号码，产生骚扰导致投诉。这两种情...,"[[0.7234765, -0.12364538, 0.8468458, 0.3685846..."


In [5]:
np.max(cosine_similarity(np.array([quality_qa_df['Q_embedding'][1][1]]), quality_qa_df['Q_embedding'][0].tolist()))


0.56324834

In [6]:
cosine_similarity([t2v_model.encode('怎么要这么多资料')], [t2v_model.encode('价格太高了')])
cosine_similarity(np.array([t2v_model.encode('怎么要这么多资料')]), np.array([t2v_model.encode('价格太高了')]))


array([[0.5632484]], dtype=float32)

## handle the raw data procession

In [ ]:
raw_data = pd.read_csv("data/raw_data.csv")

conversation_data = pd.DataFrame(columns=['id', '对话内容'])
# 遍历raw_data，将同一id的对话内容合并
for i in tqdm(range(raw_data.shape[0])):
    if i == 0:
        conversation_data.loc[0] = [
            raw_data.loc[i, '文件名'], raw_data.loc[i, '对话内容']]
    elif raw_data.loc[i-1, '对话内容'] is not np.nan:
        if raw_data.loc[i, '对话内容'] is not np.nan:
            conversation_data.loc[conversation_data.shape[0]-1, '对话内容'] = conversation_data.loc[conversation_data.shape[0] -
                                                                                                1, '对话内容'] + '\n' + raw_data.loc[i, '对话内容']
        else:
            pass
    else:
        conversation_data.loc[conversation_data.shape[0]] = [
            raw_data.loc[i, '文件名'], raw_data.loc[i, '对话内容']]

conversation_data.drop(columns=['id'], inplace=True)
conversation_data = conversation_data[1:]
conversation_data.reset_index(drop=True, inplace=True)

for i in tqdm(range(conversation_data.shape[0])):
    if pd.isnull(conversation_data.loc[i, '对话内容']) is False:
        conversation_data.loc[i, '对话内容'] = conversation_data.loc[i, '对话内容'].replace(
            '?', '\n').replace('？', '\n').replace('。', '\n')

conversation_data

In [ ]:
label_list = []

for sentence in tqdm(re.split(r'\n', conversation_data.loc[21771, '对话内容'])):
    # 如果sentence长度大于等于2
    if len(sentence) >= 2:
        temp_embedding = t2v_model.encode(sentence)
        # 判断temp_embedding和Q_embedding的相似度
        for i in range(quality_qa_df.shape[0]):
            if np.max(cosine_similarity([temp_embedding], quality_qa_df.loc[i, 'Q_embedding'])) > 0.8:
                print(sentence)
                print(i)

label_list


In [ ]:
re.split(r'\n', conversation_data.loc[21771, '对话内容'])

## new data

In [9]:
import json

with open('data/qa_data.json', 'r', encoding='utf-8') as f:
    qa_list = json.load(f)

# 将qa_list转化为dataframe
qa_df = pd.DataFrame(qa_list)
qa_df


,question,answer
0,什么系统啊？不知道,"[啊，您这边有电销的业务吗？, 嗯，电话销售啊。]"
1,我我不知道你什么呢？,"[噢，销售保外呼系统的就是防电话封号的，您还有印象吗？现在还需要吗？, 嗯，好，那打扰您了哈..."
2,啊？不需要不需要。,[嗯，好，那打扰您了哈，再见！]
3,喂？,"[唉喂，你好，, 唉，你好，我这边是一辆科技的客户经理，然后我们这边是做这个外呼防封号系统的..."
4,哪里？,[唉，你好，我这边是一辆科技的客户经理，然后我们这边是做这个外呼防封号系统的，然后，然后。]
...,...,...
11984,啊？没有了，不需要了,"[呃，是已经找到了合适的机构还是怎么？, 噢，好的好的，那打扰您了啊嗯。]"
11985,啊？不用了不用了,[噢，好的好的，那打扰您了啊嗯。]
11986,OK？,[]
11987,嗯嗯，现在我们暂时都没有在弄，你加我微信吧，好吧？好,[好，行行，我可以加一下您微信，您如果说您后期如果那个手机电话卡确实因为封号了，您可以联系我...


In [10]:
# 新增一列，定义该列均为list，用于存储该问题是否与质检问题相似
qa_df['mentioned'] = [[] for i in range(qa_df.shape[0])]

for i in tqdm(range(qa_df.shape[0])):
    temp_embedding = t2v_model.encode(qa_df.loc[i, 'question'])
    # 判断temp_embedding和Q_embedding的相似度
    for y in range(quality_qa_df.shape[0]):
        if np.max(cosine_similarity([temp_embedding], quality_qa_df.loc[y, 'Q_embedding'])) > 0.8:
            qa_df.loc[i, 'mentioned'].append(y)
            break

  0%|          | 0/11989 [00:00<?, ?it/s]

In [16]:
qa_df[qa_df['mentioned'].apply(lambda x: len(x)) > 0].to_csv(
    'data/temp_for_gpt_test.csv', index=False)